In [1]:
import pandas as pd
import numpy as np
import sentence_transformers

## Creating sentence embeddings

In [2]:
 model = sentence_transformers.SentenceTransformer("../inputData/sentence_transformer_nli/model_results") #from CoronaWhy dataset
 model.cuda();

In [3]:
import os
PATH = '/home/axhue/Documents/Kaggle_Competitions/common/inputData/v6_text/'
def str_num_sort(inp_str):
    return int(inp_str.split('_')[-1].split('.')[0])

df_pickles = [PATH + p for p in os.listdir(PATH)]
df_pickles.sort(key=str_num_sort)

mini_df = pd.read_pickle(df_pickles[0],compression="gzip")

file_ids = mini_df['paper_id'].unique()
abs_df = mini_df[mini_df['section'] == 'abstract']

In [4]:
res_embeds = model.encode(abs_df['sentence'].tolist())
res_np = np.array(res_embeds)

In [5]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from plotly import express
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [6]:
''.join(abs_df[abs_df['paper_id'] == file_ids[2]]['sentence'].values)

'Progressive multifocal leukoencephalopathy (PML) is a severely debilitating and often fatal demyelinating disease of the central nervous system (CNS) in immunosuppressed individuals caused by JC polyomavirus (JCV), a ubiquitous human pathogen.Demyelination results from lytically infected oligodendrocytes, whose clearance is impaired in the setting of depressed JCV-specific T cell-mediated CNS surveillance.Although mutations in the viral capsid and genomic rearrangements in the viral non-coding region appear to set the stage for PML in the immunosuppressed population, mechanisms of demyelination and CNS antiviral immunity are poorly understood in large part due to absence of a tractable animal model that mimics PML neuropathology in humans.Early studies using mouse polyomavirus (MPyV) in T cell-deficient mice demonstrated productive viral replication in the CNS and demyelination; however, these findings were confounded by spinal cord compression by virus-induced vertebral bone tumors.H

In [7]:
a = mini_df[mini_df['paper_id'] == file_ids[9]]

a.iloc[0]['sentence']

'Recent advances in live cell imaging of hepatoma cells'

In [9]:
''.join(abs_df[abs_df['paper_id'] == file_ids[2]]) #inspect an abstract from the dataset

'paper_idlanguagesectionsentencelemmaUMLSGGPSOTAXONCHEBIGOCLDNACELL_TYPECELL_LINERNAPROTEINDISEASECHEMICALCANCERORGANTISSUEORGANISMCELLAMINO_ACIDGENE_OR_GENE_PRODUCTSIMPLE_CHEMICALANATOMICAL_SYSTEMIMMATERIAL_ANATOMICAL_ENTITYMULTI-TISSUE_STRUCTUREDEVELOPING_ANATOMICAL_STRUCTUREORGANISM_SUBDIVISIONCELLULAR_COMPONENTPATHOLOGICAL_FORMATIONORGANISM_SUBSTANCEsentence_id'

In [10]:
scale_res = scaler.fit_transform(res_np)
pca = PCA(n_components = 0.92)
pca_res = pca.fit_transform(scale_res)

In [11]:
tsne_res = TSNE(n_components=2).fit_transform(pca_res)

In [12]:
tsne_df = pd.DataFrame(tsne_res,columns = ['x','y'])
tsne_df = tsne_df.assign(paper_id=abs_df['paper_id'].reset_index(drop=True),sentence=abs_df['sentence'].reset_index(drop=True))

In [14]:
fig = express.scatter(tsne_df[tsne_df['paper_id'].isin(file_ids[:10])],x='x',y='y',color='paper_id') #adjust isin(file_ids[:x]) to visualize more documents
fig.show()

# Clustering

## Kmeans

In [15]:
from sklearn.cluster import KMeans

In [17]:
#K means Clustering 
def doKmeans(X, nclust=2):
    model = KMeans(nclust)
    model.fit(X)
    clust_labels = model.predict(X)
    cent = model.cluster_centers_
    return (clust_labels, cent)

clust_labels, cent = doKmeans(tsne_res, 10)
kmeans = pd.DataFrame(clust_labels)

In [18]:
tsne_df = tsne_df.assign(cluster=kmeans)

In [19]:
fig = express.scatter(tsne_df,x='x',y='y',color='cluster')
fig.show()

In [20]:
tsne_df = tsne_df.assign(sentence=abs_df['sentence'].reset_index(drop=True))

In [67]:
tsne_df.iloc[9343]

x                                            60.9979
y                                           -46.0571
paper_id    ce40c2d0956a6167604ab1a0955fd47f22b55111
cluster                                            3
sentence                                          []
Name: 9343, dtype: object

## DBSCAN

In [21]:
scale_res = scaler.fit_transform(res_np)
pca = PCA(n_components = 0.94)
pca_res = pca.fit_transform(scale_res)

In [24]:
pca_res.shape

(9366, 99)

In [22]:
from sklearn.cluster import DBSCAN

In [23]:
db = DBSCAN(eps=0.1, min_samples=7).fit(pca_res)
set(db.labels_)

{-1, 0, 1, 2, 3, 4, 5}

In [24]:
tsne_res = TSNE(n_components=2).fit_transform(pca_res)

In [25]:
tsne_df = pd.DataFrame(tsne_res,columns = ['x','y'])
tsne_df = tsne_df.assign(paper_id=abs_df['paper_id'].reset_index(drop=True),cluster=db.labels_)

In [26]:
fig = express.scatter(tsne_df,x='x',y='y',color='cluster')
fig.show()